In [1]:
import apache_beam as beam

from apache_beam.runners.interactive import interactive_beam as ibeam
p = ibeam.create_pipeline()

/usr/local/google/home/ningk/workspace/beam/sdks/python/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [2]:
init_pcoll = p |  'Init Create' >> beam.Create(range(10))
squares = init_pcoll | 'Square' >> beam.Map(lambda x: x*x)
cubes = init_pcoll | 'Cube' >> beam.Map(lambda x: x**3)

# Below commented code needs portable Beam schema feature.
# from apache_beam import coders
from typing import NamedTuple
from typing import Optional
import numpy as np

class ExpNum(NamedTuple):
    init: np.int16
    square: Optional[np.int32]
    cube: np.unicode
        
class Foo(object):
    def __init__(self, x, y, z):
        self._init = x
        self._square = y
        self._cube = z

# coders.registry.register_coder(ExpNum, coders.RowCoder)
# exp = init_pcoll | 'Exp' >> beam.Map(lambda x: Foo(x, x*x if x!=x*x else None, x**3 if x!=x**3 else None)).with_output_types(ExpNum)
exp = init_pcoll | 'Exp' >> beam.Map(lambda x: Foo(x, x*x if x!=x*x else None, x**3 if x!=x**3 else None))

In [3]:
print (init_pcoll.element_type)
print (exp.element_type)
# print (exp.element_type._fields)
# print (exp.element_type._field_types)

<class 'int'>
<class '__main__.Foo'>


In [7]:
result = p.run()
result.wait_until_finish()

node [color=blue, fontcolor=blue];
"Init Create";
leaf9441 [style=invis];
Square;
leaf1848 [style=invis];
Cube;
leaf6261 [style=invis];
Exp;
leaf6776 [style=invis];
"ref_AppliedPTransform_AppliedPTransform_1/_ReadCache_init_pcoll,139921372113104,ref_PCollection_PCollection_1";
"Init Create" -> leaf9441;
Square -> leaf1848;
Cube -> leaf6261;
Exp -> leaf6776;
"ref_AppliedPTransform_AppliedPTransform_1/_ReadCache_init_pcoll,139921372113104,ref_PCollection_PCollection_1" -> Square;
"ref_AppliedPTransform_AppliedPTransform_1/_ReadCache_init_pcoll,139921372113104,ref_PCollection_PCollection_1" -> Cube;
"ref_AppliedPTransform_AppliedPTransform_1/_ReadCache_init_pcoll,139921372113104,ref_PCollection_PCollection_1" -> Exp;
}



In [5]:

t_root = p._root_transform()
for apt in (t_root.parts):
    print(apt.full_label)


Init Create
Square
Cube
Exp


In [ ]:
init_list = list(result.get(init_pcoll))
squares_list = list(result.get(squares))
cubes_list = list(result.get(cubes))
exp_list = list(result.get(exp))
print (init_list)
print (squares_list)
print (cubes_list)
print (exp_list)

In [ ]:
import flatdict
data = []
for eg in exp_list:
    data.append(flatdict.FlatDict(vars(eg)).as_dict())
print(data)

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
print(df)

In [ ]:
from bokeh.plotting import figure, show, output_notebook, curdoc
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from ipywidgets import interact
output_notebook()

source = ColumnDataSource(data=df)
plt = figure()
hover = HoverTool()
hover.tooltips = []
for col in df.columns.tolist():
    hover.tooltips.append((col, '@' + col))
plt.add_tools(hover)
plt.circle(x='_init', y='_square', source=source, size=10, color='green', legend='Square')
plt.triangle(x='_init', y='_cube', source=source, size=10, color='blue', legend='Cube')
show(plt)

In [ ]:
import numpy as np
import time

def new_data():
    index = 9
    for r in np.arange(0, 2*np.pi, 0.1):
        index += 1
        sin_r = np.sin(r).astype(np.float32)
        yield {'index' : [index],
               '_init' : [sin_r], 
               '_square' : [sin_r*sin_r], 
               '_cube': [sin_r**3]}

x = new_data()

In [ ]:
from bokeh.client import push_session

def update():
    source.stream(next(x))
    time.sleep(1)

session = push_session(curdoc())
curdoc().add_periodic_callback(update, 500)
session.show()

In [ ]:
import pandas as pd
from flatten_dict import flatten
from bokeh.models import ColumnDataSource

def _key_reducer(k1, k2):
  if k1 is None:
    return str(k2)
  return str(k1) + '/' + str(k2)

x = {
    'X': [1,2,3],
    'A': {
        'Y': [4,5,6],
        'Z': [7,8,9],
        'M': (10,11,12),
        'N': [13, 14]
    }
}

y = flatten(x, reducer=_key_reducer)
print(y)

# df_y = pd.DataFrame(y)
# print(df_y)

x = [1,2,3]
y = flatten(x, reducer='path', enumerate_types=(list,tuple))
print(y)
print(type(x))
print(type(x) in (list, tuple))

x = (1,2,3)
y = flatten(x, reducer='path', enumerate_types=(list,tuple))
print(y)
print(type(x))
print(type(x) in (list, tuple))

x = { 'X': [{'1': 1}, {'2':2}, {'3':3}]}
y = flatten(x, reducer=_key_reducer)
print(y)
print(type(x))
print(type(x) in (list, tuple))


In [ ]:
class Bar(object):
    def __init__(self):
        self.bx = 1


class Foo(object):
    def __init__(self):
        self.fx = Bar()
        
        
print(vars(Foo()))

import json
s = json.dumps(vars(Foo()))

In [8]:
import pandas as pd

data = {
  'name': ['a', 'b', 'c'],
  'type': ['A', 'B', 'C'],
  'qty': [1, 2, 3],
  'price': [4, 5, 6],
}

print (data)

df = pd.DataFrame(data, index = ['x','1','1'])

print(df)
df.iloc[0]
df.loc['1']

{'name': ['a', 'b', 'c'], 'type': ['A', 'B', 'C'], 'qty': [1, 2, 3], 'price': [4, 5, 6]}
  name type  qty  price
x    a    A    1      4
1    b    B    2      5
1    c    C    3      6


,name,type,qty,price
1,b,B,2,5
1,c,C,3,6


In [ ]:
from bokeh.models.tools import HoverTool

def auto_populate_hover(df):
  hover = HoverTool()
  hover.tooltips = []
  for col in df.columns.tolist():
    hover.tooltips.append((col, '@' + col))
  return hover

In [ ]:
from bokeh.plotting import figure, show, output_notebook, curdoc
from bokeh.models import ColumnDataSource

output_notebook()

source = ColumnDataSource(data=df)
x1 = source.data['name'].tolist()
x2 = source.data['index'].tolist()
x = ['.'.join(item) for item in zip(x1, x2)]
column_name = source.add(x)
plt = figure(x_range=x)
plt.add_tools(auto_populate_hover(df))
plt.circle(x=column_name, y='price', source=source, size=10, color='green', legend = 'Price by Name')
plt.line(x=column_name, y='price', source=source)
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral5
color_map = factor_cmap(field_name=column_name, palette=Spectral5, factors=x)
plt.vbar(x=column_name, top='price', source=source, width=0.70, color=color_map)
show(plt)